## Reflection

- This plot, arguably, contains all the information you need, and in theory a static version could be produced for a report or paper by selecting a certain, clear angle.
- However, is it very easy to interpret?
- Transparency is a bit strange - some parts are more opaque than others, and this can be a bit confusing.
- Merging two spatial dimensions and a time dimension into one object is a bit confusing... would the reviewer recognise this immediately?

- AND now it's not trivial to compare to other variables in the same place! It suddenly becomes a lot more difficult to compare two variables in the same place, as you would need to produce two separate 3D plots and then try to compare them mentally.

- This is a good demonstration of how tricky spatial data can be to visualise effectively!